In [ ]:
import os
import numpy as np
import pandas as pd
import csv
import time

from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/Data")

!pip install transformers
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline('sentiment-analysis', model=pt_model, tokenizer=tokenizer)

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
def get_senti_labels(content):
    result = classifier(content)[0]
    if result['label'] == "NEGATIVE":
        label = 0
    else:
        label = 1
    return label, round(result['score'], 4)

In [ ]:
df_chunk = pd.read_csv('textid.csv', header = None, dtype = str, \
                                            chunksize=300000, encoding = 'utf-8-sig', engine='python')
file_counts = 0
for chunk in df_chunk:
    chunk_time = time.time()
    chunk.columns = ['text', 'text_id']
    chunk.dropna(subset=["text_id"], axis=0, how='any', inplace=True)
    chunk.dropna(subset=['text'], axis=0, how='any', inplace=True)
    
    chunk.loc[:, "st_labels"] = chunk["text"].astype(str).apply(lambda r: get_senti_labels(r)[0])
    chunk.loc[:, "st_scores"] = chunk["text"].astype(str).apply(lambda r: get_senti_labels(r)[1])

    chunk.to_csv('textid_st.csv', mode='a', header=None, index=False, encoding = 'utf-8-sig') #"text", "text_id", "st_labels", "st_scores"
    
    file_counts += 1
    print(f"File {file_counts}:")
    print(f"- cost time: {(time.time() - chunk_time)//60} minutes")